# Web Scraping Notebook
## NBA Player Career Stats

In [1]:
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
import pandas as pd
import time
import requests

In [2]:
f = open("players.txt", "r")
ply_list = f.read().split("\n")
print(ply_list)

['Jordan, Michael', 'Durant, Kevin', 'Iverson, Allen', 'Gervin, George', 'Malone, Karl', 'Bryant, Kobe', 'Wilkins, Dominique', 'Barry, Rick', 'Abdul-Jabbar, Kareem', 'Dantley, Adrian', 'Bird, Larry', 'Erving, Julius', 'Maravich, Peter', 'Thompson, David', 'Issel, Dan', 'King, Bernard', 'Barkley, Charles', 'McAdoo, Bob', 'Wade, Dwyane', 'Somerset, Willie', 'Olajuwon, Hakeem', 'English, Alex', 'Cunningham, Billy', 'Cousins, DeMarcus', 'Robinson, David', 'Hayes, Elvin', 'Ewing, Patrick', 'Richmond, Mitch', 'Nowitzki, Dirk', 'Webber, Chris', 'Drew, John', 'Scott, Charlie', 'Robinson III, Glenn', 'Arenas, Gilbert', 'Brisker, John', 'Drexler, Clyde', 'Malone, Moses', 'Free, World', 'Haywood, Spencer', 'McGinnis, George', 'Verga, Bob', 'Lanier, Bob', 'Johnson, Magic', 'Johnson, Marques', 'Aguirre, Mark', 'Mitchell, Mike', 'Pierce, Paul', 'Vandeweghe, Kiki', 'McGrady, Tracy', 'Thompson, Klay', 'Marbury, Stephon', 'Thomas, Isaiah', 'Thomas, Isiah', 'Bosh, Chris', 'Mashburn, Jamal', 'Jones, Larr

In [3]:
root = "https://www.basketball-reference.com/players/"
players = pd.DataFrame({"fname":[],
                        "lname":[],
                        "g":[],
                        "pts":[],
                        "trb":[],
                        "ast":[],
                        "fgp":[],
                        "fg3p":[],
                        "ftp":[],
                        "efgp":[],
                        "per":[],
                        "ws":[],
                         })
players.head()

,fname,lname,g,pts,trb,ast,fgp,fg3p,ftp,efgp,per,ws


In [4]:
# Ending pattern = 'last initial' + / + 'last[:5]' + 'first[:2]' + '01.html'
# Eg. j/jordami01.html

def rooter(item):
    sub = item.split(",")
    add = str(sub[0][0]) + "/" + str(sub[0][:5]) +  str(sub[1][:3]) + "01.html"
    add = add.replace(" ","").lower()
    return(str(root + add))
print(rooter('Stojakovic, Peja'))

https://www.basketball-reference.com/players/s/stojape01.html


In [5]:
def plyr_scrape(name, y):
    url = rooter(name)
    name = name.split(", ")
    fname = name[1]
    lname = name[0]
    headers = requests.utils.default_headers()
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    page = soup.prettify()
    page1 = soup.findAll("div", {"class": "p1"})
    page2 = soup.findAll("div", {"class": "p2"})
    page3 = soup.findAll("div", {"class": "p3"})
    x = 1
    page1 = str(page1).split("</div>")
    page2 = str(page2).split("</div>")
    page3 = str(page3).split("</div>")
    for n in page1:
        if "Games" in n:
            n = str(n).split("</h4><p></p>")
            g = n[1]
            g = str(g).replace("<p>","")
            g = str(g).replace("</p>","")
            g = str(g).replace("\n","")
        elif "Points" in n:
            n = str(n).split("</h4><p></p>")
            pts = n[1]
            pts = str(pts).replace("<p>","")
            pts = str(pts).replace("</p>","")
            pts = str(pts).replace("\n","")
        elif "Total Rebounds" in n:
            n = str(n).split("</h4><p></p>")
            trb = n[1]
            trb = str(trb).replace("<p>","")
            trb = str(trb).replace("</p>","")
            trb = str(trb).replace("\n","")
        elif "Assists" in n:
            n = str(n).split("</h4><p></p>")
            ast = n[1]
            ast = str(ast).replace("<p>","")
            ast = str(ast).replace("</p>","")
            ast = str(ast).replace("\n","")
    x = 1
    for n in str(page2).split("</h4><p></p>"):
        if x == 1:
            x += 1
            continue
        elif x == 2:            
            p = str(n).split("</p>',")
            fgp = p[0]
            fgp = str(fgp)[5:]
            x += 1
            continue
        elif x == 3:            
            p = str(n).split("</p>',")
            fg3p = p[0]
            fg3p = str(fg3p)[5:]
            x += 1
            continue
        elif x == 4:
            x += 1
            p = str(n).split("</p>',")
            ftp = p[0]
            ftp = str(ftp)[5:]
            continue
        elif x == 5:
            x += 1
            p = str(n).split("</p>',")
            efgp = p[0]
            efgp = str(efgp)[5:]
            continue
    x = 1
    for n in str(page3).split("</h4><p></p>"):
        if x == 1:
            x += 1
            continue
        elif x == 2:
            x += 1
            p = str(n).split("</p>',")
            per = p[0]
            per = str(per)[5:]
        elif x == 3:
            ws = n.split("</p>")
            ws = str(ws[0])[5:]
            x += 1
            continue
    df = pd.DataFrame({"fname":[fname],
                        "lname":[lname],
                        "g":[g],
                        "pts":[pts],
                        "trb":[trb],
                        "ast":[ast],
                        "fgp":[fgp],
                        "fg3p":[fg3p],
                        "ftp":[ftp],
                        "efgp":[efgp],
                        "per":[per],
                        "ws":[ws],
                         },index = [y])
    return df
    
print(plyr_scrape(ply_list[0], 1))

     fname   lname     g   pts  trb  ast   fgp  fg3p   ftp  efgp   per     ws
1  Michael  Jordan  1072  30.1  6.2  5.3  49.7  32.7  83.5  50.9  27.9  214.0


In [ ]:
y = 1
for player in ply_list:
    try:
        insert = plyr_scrape(player, y)
    except:
        continue
    players = players.append(insert)
    y += 1
players.head()

In [ ]:
players.head(400)

In [ ]:
players.to_csv("scrape.csv")

In [9]:
def rookie(name, y):
    url = rooter(name)
    name = name.split(", ")
    fname = name[1]
    lname = name[0]
    headers = requests.utils.default_headers()
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    page = soup.prettify()
    height = soup.findAll("span",{"itemprop":"height"})
    weight = soup.findAll("span",{"itemprop":"weight"})
    birth = soup.findAll("span", {"itemprop":"birthDate"})
    b_place = soup.findAll("span", {"itemprop":"birthPlace"})
    href = soup.findAll("a")
    p = soup.findAll("p")
    
    height = str(height).split('"height">')
    height = str(height[1]).replace("</span>]","")
    
    weight = str(weight).split('"weight">')
    weight = str(weight[1]).replace("</span>]","")
    weight = weight[:5]
    
    b_place = str(b_place).split('"birthPlace">')
    b_place = b_place[1]
    b_place = str(b_place).split(">")
    
    city = str(b_place[0]).split(",")
    city = str(city[0]).replace("in", "")
    city = city.strip()
    
    state = b_place[1]
    state = str(state).split("<")[0]
    
    for i in href:
        if "/friv/colleges.fcgi?" in str(i):
            college = i
            college = str(college).split(">")
            college = str(college[1]).replace("</a","")
        elif "/boxscores/" in str(i) and '"/boxscores/"' not in str(i):
            debut = i
            debut = str(debut).split(">")
            debut = str(debut[1]).replace("</a","")
            debut = debut[-4:]
    
    df = pd.DataFrame({"fname":[fname],
                        "lname":[lname],
                        "school":[college],
                        "debut":[debut],
                        "ht":[height],
                        "wt":[weight],
                        "b_city":[city],
                        "b_state":[state],
                         },index = [y])
    return df
    
print(rookie(ply_list[0], 1))


     fname   lname school debut   ht     wt    b_city   b_state
1  Michael  Jordan    UNC  1984  6-6  195lb  Brooklyn  New York


In [10]:
meta = pd.DataFrame({"fname":[],
                      "lname":[],
                      "school":[],
                      "debut":[],
                      "ht":[],
                      "wt":[],
                      "b_city":[],
                      "b_state":[],
                       })
meta.head()

,fname,lname,school,debut,ht,wt,b_city,b_state


In [12]:
y = 1
for player in ply_list:
    try:
        insert = rookie(player, y)
    except:
        player = str(player).split(",")
        df = pd.DataFrame({"fname":[player[1]],
                        "lname":[player[0]],
                        "school":[""],
                        "debut":[""],
                        "ht":[""],
                        "wt":[""],
                        "b_city":[""],
                        "b_state":[""],
                         },index=[y])
        meta = meta.append(df)
        y += 1
        continue
    meta = meta.append(insert)
    y += 1
meta.head()

IndexError: list index out of range

In [14]:
meta.head(2000)

,fname,lname,school,debut,ht,wt,b_city,b_state
1,Michael,Jordan,UNC,1984,6-6,195lb,Brooklyn,New York
2,Kevin,Durant,Texas,LAL,6-10,240lb,Washgton,District of Columbia
3,Allen,Iverson,Georgetown,1996,6-0,165lb,Hampton,Virginia
4,George,Gervin,Eastern Michigan,1973,6-7,180lb,Detroit,Michigan
5,Karl,Malone,Louisiana Tech,1985,6-9,250lb,Summerfield,Louisiana
1,Michael,Jordan,UNC,1984,6-6,195lb,Brooklyn,New York
2,Kevin,Durant,Texas,LAL,6-10,240lb,Washgton,District of Columbia
3,Allen,Iverson,Georgetown,1996,6-0,165lb,Hampton,Virginia
4,George,Gervin,Eastern Michigan,1973,6-7,180lb,Detroit,Michigan
5,Karl,Malone,Louisiana Tech,1985,6-9,250lb,Summerfield,Louisiana


In [17]:
scrape = pd.read_csv("scrape.csv")
scrape.head(20)

,fname,lname,g,pts,trb,ast,fgp,fg3p,ftp,efgp,per,ws
0,Alaa,Abdelnaby,256,5.7,3.3,0.3,50.2,0,70.1,50.2,13,4.8
1,Kareem,Abdul-Jabbar,1560,24.6,11.2,3.6,55.9,5.6,72.1,55.9,24.6,273.4
2,Tariq,Abdul-Wahad,236,7.8,3.3,1.1,41.7,23.7,70.3,42.2,11.4,3.5
3,Shareef,Abdur-Rahim,830,18.1,7.5,2.5,47.2,29.7,81,47.9,19,71.2
4,Tom,Abernethy,319,5.6,3.2,1.2,49.2,0,74.7,49.2,12.9,13.4
5,Alex,Abrines,174,5.3,1.4,0.5,38.7,36.8,88,52.5,8.8,5.0
6,Alex,Acker,30,2.7,1.0,0.5,37,32,50,41.3,8.5,-0.2
7,Mark,Acres,375,3.6,4.1,0.5,50.6,53.8,66.5,50.9,9,9.3
8,Quincy,Acy,337,4.9,3.5,0.6,44.4,35,75.9,51.3,11.2,8.7
9,Alvan,Adams,988,14.1,7.0,4.1,49.8,13.3,78.8,49.8,18.3,73.5


In [21]:
ult = pd.merge(scrape, meta,  how='outer', on=['fname','lname'])
ult.head(20)

,fname,lname,g,pts,trb,ast,fgp,fg3p,ftp,efgp,per,ws,school,debut,ht,wt,b_city,b_state
0,Alaa,Abdelnaby,256.0,5.7,3.3,0.3,50.2,0,70.1,50.2,13,4.8,Duke,1990,6-10,240lb,Cairo,Egypt
1,Kareem,Abdul-Jabbar,1560.0,24.6,11.2,3.6,55.9,5.6,72.1,55.9,24.6,273.4,UCLA,1969,7-2,225lb,New York,New York
2,Tariq,Abdul-Wahad,236.0,7.8,3.3,1.1,41.7,23.7,70.3,42.2,11.4,3.5,San Jose State,1997,6-6,223lb,Maisons Alfort,France
3,Shareef,Abdur-Rahim,830.0,18.1,7.5,2.5,47.2,29.7,81,47.9,19,71.2,California,1996,6-9,225lb,Marietta,Georgia
4,Tom,Abernethy,319.0,5.6,3.2,1.2,49.2,0,74.7,49.2,12.9,13.4,Indiana,1976,6-7,220lb,South Bend,Indiana
5,Alex,Abrines,174.0,5.3,1.4,0.5,38.7,36.8,88,52.5,8.8,5.0,NaN,NaN,NaN,NaN,NaN,NaN
6,Alex,Acker,30.0,2.7,1.0,0.5,37,32,50,41.3,8.5,-0.2,Pepperdine,2006,6-5,185lb,Compton,California
7,Mark,Acres,375.0,3.6,4.1,0.5,50.6,53.8,66.5,50.9,9,9.3,Oral Roberts,1987,6-11,220lb,Inglewood,California
8,Quincy,Acy,337.0,4.9,3.5,0.6,44.4,35,75.9,51.3,11.2,8.7,Baylor,2012,6-7,240lb,Tyler,Texas
9,Alvan,Adams,988.0,14.1,7.0,4.1,49.8,13.3,78.8,49.8,18.3,73.5,Oklahoma,1975,6-9,210lb,Lawrence,Kansas


In [22]:
ult.to_csv("scrape2.csv")

In [29]:
f = open("players2.txt", "r")
list2 = f.read().split("\n")
print(list2)

['Abrines, Alex', 'Ajinca, Alexis', 'Aldemir, Furkan', 'Andriuskevicius, Martynas', 'Anstey, Chris', 'Antic, Pero', 'Asik, Omer', 'Ayon, Gustavo', 'Bagaric, Dalibor', 'Barbosa, Leandro', 'Bargnani, Andrea', 'Bateer, Mengke', 'Batista, Esteban', 'Bazarevich, Sergei', 'Beaubois, Rodrigue', 'Bender, Jonathan', 'Biedrins, Andris', 'Blatche, Andray', 'Borrell, Lazaro', 'Bradtke, Mark', 'Brezec, Primoz', 'Brown, Kwame', 'Brussino, Nicolas', 'Bryant, Kobe', 'Butler, Jackie', 'Bynum, Andrew', 'Cabarkapa, Zarko', 'Calderon, Jose', 'Casspi, Omri', 'Claver, Victor', 'Creek, Mitchell', 'Curcic, Radisav', 'Curry, Eddy', 'Cvetkovic, Rastko', 'Danilovic, Sasha', 'David, Kornel', 'Dawkins, Darryl', 'Delfino, Carlos', 'Diaw, Boris', 'Diop, DeSagana', 'Divac, Vlade', 'Djordjevic, Aleksandar', 'Dragic, Zoran', 'Drobnjak, Predrag', 'Ebi, Ndudi', 'Ellis, Monta', 'Erden, Semih', 'Esposito, Vincenzo', 'Eyenga, Christian', 'Faverani, Vitor', 'Fernandez, Rudy', 'Fesenko, Kyrylo', 'Fotsis, Antonis', 'Freeland, 

In [30]:
meta2 = pd.DataFrame({"fname":[],
                      "lname":[],
                      "debut":[],
                      "ht":[],
                      "wt":[],
                      "b_city":[],
                      "b_state":[],
                       })
meta2.head()

,fname,lname,debut,ht,wt,b_city,b_state


In [31]:
def rookie2(name, y):
    url = rooter(name)
    name = name.split(", ")
    fname = name[1]
    lname = name[0]
    headers = requests.utils.default_headers()
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    page = soup.prettify()
    height = soup.findAll("span",{"itemprop":"height"})
    weight = soup.findAll("span",{"itemprop":"weight"})
    birth = soup.findAll("span", {"itemprop":"birthDate"})
    b_place = soup.findAll("span", {"itemprop":"birthPlace"})
    href = soup.findAll("a")
    p = soup.findAll("p")
    
    height = str(height).split('"height">')
    height = str(height[1]).replace("</span>]","")
    
    weight = str(weight).split('"weight">')
    weight = str(weight[1]).replace("</span>]","")
    weight = weight[:5]
    
    b_place = str(b_place).split('"birthPlace">')
    b_place = b_place[1]
    b_place = str(b_place).split(">")
    
    city = str(b_place[0]).split(",")
    city = str(city[0]).replace("in", "")
    city = city.strip()
    
    state = b_place[1]
    state = str(state).split("<")[0]
    
    for i in href:
        if "/boxscores/" in str(i) and '"/boxscores/"' not in str(i):
            debut = i
            debut = str(debut).split(">")
            debut = str(debut[1]).replace("</a","")
            debut = debut[-4:]
    
    df = pd.DataFrame({"fname":[fname],
                        "lname":[lname],
                        "debut":[debut],
                        "ht":[height],
                        "wt":[weight],
                        "b_city":[city],
                        "b_state":[state],
                         },index = [y])
    return df
    
print(rookie2("Bryant, Kobe", 1))


  fname   lname debut   ht     wt        b_city       b_state
1  Kobe  Bryant  1996  6-6  212lb  Philadelphia  Pennsylvania


In [32]:
y = 1
for player in list2:
    try:
        insert = rookie2(player, y)
    except:
        continue
    meta2 = meta2.append(insert)
    y += 1
meta2.head()

,fname,lname,debut,ht,wt,b_city,b_state
1,Alex,Abrines,2016,6-6,200lb,Palma de Mallorca,Spain
2,Alexis,Ajinca,2008,7-2,248lb,Sat-Etienne,France
3,Furkan,Aldemir,2014,6-10,240lb,Konak,Turkey
4,Martynas,Andriuskevicius,2006,7-2,240lb,Kaunas,Lithuania
5,Chris,Anstey,1997,7-0,249lb,Melbourne,Australia


In [33]:
scrape2 = pd.read_csv("scrape2.csv")
scrape2.head(20)

,fname,lname,school,debut,wt,ht(inches),b_city,b_state,g,pts,trb,ast,fgp,fg3p,ftp,efgp,per,ws,Unnamed: 18
0,Manute,Bol,University of Bridgeport,1985,200lb,91.0,Gogrial,South Sudan,624.0,2.6,4.2,0.3,40.7,21,56.1,42,9.4,12.3,NaN
1,Shawn,Bradley,BYU,1993,235lb,90.0,Landstuhl,Germany,832.0,8.1,6.3,0.7,45.7,10.3,71.6,45.7,16,40.7,NaN
2,Sim,Bhullar,New Mexico State,2015,360lb,89.0,Ontario,Canada,3.0,0.7,0.3,0.3,50,-,-,50,44.3,0.0,NaN
3,Chuck,Nevitt,NC State,1983,217lb,89.0,Cortez,Colorado,155.0,1.6,1.5,0.1,43.8,-,58.9,43.8,9.1,0.5,NaN
4,Mark,Eaton,UCLA,1982,275lb,88.0,Westmster,California,875.0,6.0,7.9,1.0,45.8,0,64.9,45.8,10.9,44.8,NaN
5,Priest,Lauderdale,Central State University,1996,325lb,88.0,Chicago,Illinois,74.0,3.4,1.9,0.4,47.2,0,55.4,47.2,11.3,-0.8,NaN
6,Ralph,Sampson,Virginia,1983,228lb,88.0,Harrisonburg,Virginia,456.0,15.4,8.8,2.3,48.6,17.2,66.1,48.7,16,20.1,NaN
7,Rik,Smits,Marist College,1988,250lb,88.0,Edhoven,Netherlands,867.0,14.8,6.1,1.4,50.7,11.5,77.3,50.7,17.9,56.6,NaN
8,Randy,Breuer,Minnesota,1983,230lb,87.0,Lake City,Minnesota,681.0,6.8,4.4,0.9,46.7,0,62.8,46.7,13.2,20.9,NaN
9,Keith,Closs,Central Connecticut State University,1997,212lb,87.0,Hartford,Connecticut,130.0,3.9,2.9,0.3,47.1,0,60.6,47.1,14.2,1.9,NaN


In [34]:
ult2 = pd.merge(scrape2, meta2,  how='outer', on=['fname','lname'])
ult.head(20)

,fname,lname,g,pts,trb,ast,fgp,fg3p,ftp,efgp,per,ws,school,debut,ht,wt,b_city,b_state
0,Alaa,Abdelnaby,256.0,5.7,3.3,0.3,50.2,0,70.1,50.2,13,4.8,Duke,1990,6-10,240lb,Cairo,Egypt
1,Kareem,Abdul-Jabbar,1560.0,24.6,11.2,3.6,55.9,5.6,72.1,55.9,24.6,273.4,UCLA,1969,7-2,225lb,New York,New York
2,Tariq,Abdul-Wahad,236.0,7.8,3.3,1.1,41.7,23.7,70.3,42.2,11.4,3.5,San Jose State,1997,6-6,223lb,Maisons Alfort,France
3,Shareef,Abdur-Rahim,830.0,18.1,7.5,2.5,47.2,29.7,81,47.9,19,71.2,California,1996,6-9,225lb,Marietta,Georgia
4,Tom,Abernethy,319.0,5.6,3.2,1.2,49.2,0,74.7,49.2,12.9,13.4,Indiana,1976,6-7,220lb,South Bend,Indiana
5,Alex,Abrines,174.0,5.3,1.4,0.5,38.7,36.8,88,52.5,8.8,5.0,NaN,NaN,NaN,NaN,NaN,NaN
6,Alex,Acker,30.0,2.7,1.0,0.5,37,32,50,41.3,8.5,-0.2,Pepperdine,2006,6-5,185lb,Compton,California
7,Mark,Acres,375.0,3.6,4.1,0.5,50.6,53.8,66.5,50.9,9,9.3,Oral Roberts,1987,6-11,220lb,Inglewood,California
8,Quincy,Acy,337.0,4.9,3.5,0.6,44.4,35,75.9,51.3,11.2,8.7,Baylor,2012,6-7,240lb,Tyler,Texas
9,Alvan,Adams,988.0,14.1,7.0,4.1,49.8,13.3,78.8,49.8,18.3,73.5,Oklahoma,1975,6-9,210lb,Lawrence,Kansas


In [35]:
ult2.to_csv("scrape.csv")